In [23]:
import tensorflow as tf
import string
import requests

In [24]:
file_path = "/kaggle/input/shakespeare1-txt/shakespeare1.txt"
with open(file_path, "r", encoding="utf-8") as file1:
    response = file1.read()

In [25]:
response[0]

'T'

In [26]:
data = response.split('\n')
data[0]

'This is the 100th Etext file presented by Project Gutenberg, and'

In [27]:
data = data[253:]

In [28]:
data[0]

'  From fairest creatures we desire increase,'

In [29]:
len(data)

124204

In [30]:
data = " ".join(data)

In [31]:
data[100]

't'

In [32]:
def cleantext(doc):
  tokens = doc.split()
  table = str.maketrans('','',string.punctuation)
  tokens = [w.translate(table) for w in tokens]
  tokens = [word for word in tokens if word.isalpha()]
  tokens = [word.lower() for word in tokens]
  return tokens

In [33]:
tokens = cleantext(data)
print(tokens[:50])

['from', 'fairest', 'creatures', 'we', 'desire', 'increase', 'that', 'thereby', 'beautys', 'rose', 'might', 'never', 'die', 'but', 'as', 'the', 'riper', 'should', 'by', 'time', 'decease', 'his', 'tender', 'heir', 'might', 'bear', 'his', 'memory', 'but', 'thou', 'contracted', 'to', 'thine', 'own', 'bright', 'eyes', 'feedst', 'thy', 'lights', 'flame', 'with', 'selfsubstantial', 'fuel', 'making', 'a', 'famine', 'where', 'abundance', 'lies', 'thy']


In [34]:
len(tokens)

898199

In [35]:
len(set(tokens)) #unique words

27956

In [36]:
length = 50 + 1
lines = []

for i in range(length, len(tokens)):
  seq = tokens[i - length:i]
  line = ' '.join(seq)
  lines.append(line)
  if i > 200000:
    break

print(len(lines))

199951


In [37]:
lines[0]

'from fairest creatures we desire increase that thereby beautys rose might never die but as the riper should by time decease his tender heir might bear his memory but thou contracted to thine own bright eyes feedst thy lights flame with selfsubstantial fuel making a famine where abundance lies thy self'

In [38]:
tokens[50]

'self'

In [39]:
lines[1]

'fairest creatures we desire increase that thereby beautys rose might never die but as the riper should by time decease his tender heir might bear his memory but thou contracted to thine own bright eyes feedst thy lights flame with selfsubstantial fuel making a famine where abundance lies thy self thy'

## Build LSTM Model and Prepare X & Y

In [40]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [41]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [42]:
sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:, -1]

In [43]:
X[0]

array([   47,  1408,  1264,    37,   451,  1406,     9,  2766,  1158,
        1213,   171,   132,   269,    20,    24,     1,  4782,    87,
          30,    98,  4781,    18,   715,  1263,   171,   211,    18,
         829,    20,    27,  3807,     4,   214,   121,  1212,   153,
       13004,    31,  2765,  1847,    16, 13003, 13002,   754,     7,
        3806,    99,  2430,   466,    31])

In [44]:
y[0]

307

In [45]:
vocab_size = len(tokenizer.word_index) + 1

In [46]:
y = to_categorical(y, num_classes = vocab_size)
X.shape[1]

50

In [47]:
seq_length = X.shape[1]

## LSTM Model

In [48]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length = seq_length))
model.add(LSTM(50, return_sequences = True))
model.add(LSTM(100))
model.add(Dense(100, activation = 'relu'))
model.add(Dense(vocab_size, activation = 'softmax'))

In [49]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            650450    
                                                                 
 lstm (LSTM)                 (None, 50, 50)            20200     
                                                                 
 lstm_1 (LSTM)               (None, 100)               60400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 13009)             1313909   
                                                                 
Total params: 2055059 (7.84 MB)
Trainable params: 2055059 (7.84 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [51]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [30]:
# model.fit(X, y, batch_size = 512, epochs = 10)

In [30]:
# model.fit(X, y, batch_size = 512, epochs = 20)

Epoch 1/20
391/391 [==============================] - 67s 150ms/step - loss: 6.9716 - accuracy: 0.0281
Epoch 2/20
391/391 [==============================] - 25s 63ms/step - loss: 6.6133 - accuracy: 0.0358
Epoch 3/20
391/391 [==============================] - 18s 46ms/step - loss: 6.3875 - accuracy: 0.0531
Epoch 4/20
391/391 [==============================] - 16s 40ms/step - loss: 6.1302 - accuracy: 0.0741
Epoch 5/20
391/391 [==============================] - 15s 39ms/step - loss: 5.9314 - accuracy: 0.0884
Epoch 6/20
391/391 [==============================] - 15s 37ms/step - loss: 5.7719 - accuracy: 0.0968
Epoch 7/20
391/391 [==============================] - 15s 38ms/step - loss: 5.6343 - accuracy: 0.1038
Epoch 8/20
391/391 [==============================] - 15s 38ms/step - loss: 5.5047 - accuracy: 0.1088
Epoch 9/20
391/391 [==============================] - 14s 36ms/step - loss: 5.3814 - accuracy: 0.1136
Epoch 10/20
391/391 [==============================] - 14s 36ms/step - loss: 5.25

In [52]:
model.fit(X, y, batch_size = 512, epochs = 30)

391/391 [==============================] - 65s 143ms/step - loss: 7.0002 - accuracy: 0.0286
Epoch 2/30
391/391 [==============================] - 25s 63ms/step - loss: 6.6495 - accuracy: 0.0351
Epoch 3/30
391/391 [==============================] - 18s 46ms/step - loss: 6.4912 - accuracy: 0.0438
Epoch 4/30
391/391 [==============================] - 15s 40ms/step - loss: 6.3627 - accuracy: 0.0497
Epoch 5/30
391/391 [==============================] - 13s 34ms/step - loss: 6.2490 - accuracy: 0.0586
Epoch 6/30
391/391 [==============================] - 13s 34ms/step - loss: 6.1308 - accuracy: 0.0674
Epoch 7/30
391/391 [==============================] - 13s 32ms/step - loss: 6.0226 - accuracy: 0.0772
Epoch 8/30
391/391 [==============================] - 13s 34ms/step - loss: 5.9310 - accuracy: 0.0852
Epoch 9/30
391/391 [==============================] - 13s 32ms/step - loss: 5.8482 - accuracy: 0.0904
Epoch 10/30
391/391 [==============================] - 12s 31ms/step - loss: 5.7721 - accura

In [53]:
seed_text="From fairest creatures we desire increase"
seed_text

'From fairest creatures we desire increase'

In [54]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([seed_text])[0]
        encoded = pad_sequences([encoded], maxlen=text_seq_length, truncating='pre')
        
        # Use predict instead of predict_classes
        y_predict = model.predict(encoded)
        
        predicted_word_index = np.argmax(y_predict)
        predicted_word = ""
        
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                predicted_word = word
                break

        seed_text += " " + predicted_word

    print(seed_text)


In [55]:
generate_text_seq(model, tokenizer, seq_length, seed_text, 100)

1/1 [==============================] - 0s 23ms/step
From fairest creatures we desire increase a the a sable absurd oblivion and the world is the king of the world and the main world is dead and deeper as unfathered as lovely as the proudest sail idiot scene the squire are the public anatomy a pleasant a dullard in the world and i have seen the king of this a scarre that i have not the capitol of rome and of the fiends and i have been a man and a man and i had not donet to say i am not of the world and i had not donet to know you to be


It's accuracy has been increased as we increased the size of epochos from 10 to 20
